In [1]:
import os
from time import sleep
from tkinter import *
from PIL import ImageGrab, Image
import tkinter.messagebox

root_width =  250
root_height = 400
geo_str = str(root_width) + "x" + str(root_height)

In [2]:
import csv
#加载数据
csvPath = "color2.csv"
def toRGBStr(R,G,B):
    return "#" + toHs2(R) +  toHs2(G) + toHs2(B)
def toHs2(n):
    str = hex(n)[2:]
    if len(str)== 1:
        str = "0"+ str
    return str
class Color:
    def __init__(self, rgb, chinese):
        self.rgb = rgb
        self.chn = chinese
        self.R, self.G, self.B = int(rgb[1:3],16), int(rgb[3:5],16), int(rgb[5:7],16)
    def __repr__(self):
        return "R:" + str(self.R) + " G:"+ str(self.G) + " B" + str(self.B)
    def distance(self, other):
        return (other.R - self.R)**2 + (other.G - self.G)** 2 + (other.B - self.B)**2
    def counter(self):
        return toRGBStr(255-self.R,255-self.G,255-self.B)
    def compare(self):
        return (toRGBStr(255-self.R,255-self.G,self.B),toRGBStr(self.R,255-self.G,255-self.B),toRGBStr(255-self.R,self.G,255-self.B))
colorTab = []
with open(csvPath,encoding = "utf-8")as f:
    f_csv = csv.reader(f)
    next(f_csv)
    colorTab = [Color(row[0], row[1]) for row in f_csv]
    for row in f_csv:
        colorTab.append(Color(row[0],row[1]))
    #print(colorTab[0:5])


In [3]:
class CaptureWindow:
    def __init__(self, picPath):
        self.X = IntVar(value=0)
        self.Y = IntVar(value=0)
         # 获取屏幕尺寸
        screenWidth = root.winfo_screenwidth()
        screenHeight = root.winfo_screenheight()

         # 创建顶级组件容器，与屏幕尺寸一样大
        self.top = Toplevel(root, width=screenWidth, height=screenHeight)

         # 不显示最大化、最小化按钮
        #self.top.overrideredirect(True)
        self.image = PhotoImage(file=picPath)
         # 创建画布
        self.canvas = Canvas(self.top,bg='white', width=screenWidth, height=screenHeight)
         # 显示全屏截图，在全屏截图上进行区域截图
        self.canvas.create_image(screenWidth//2, screenHeight//2, image=self.image)
        def onLeftButtonUp(event):
            im = Image.open(picPath)
            color = im.getpixel((event.x, event.y))[:3]
            color = map(lambda x:hex(x)[2:], color)
            color = map(lambda x:x if len(x)==2 else '0'+x, color)
            color = '#' + ''.join(color)
            colorObj = Color(color, "anything")
            closest = None
            minDis = 3 * 255 ** 2
            for c in colorTab:
                curDis = colorObj.distance(c)
                if  curDis < minDis:
                    closest = c
                    minDis = curDis
            #print("minDis" + str(minDis))
            yourRGB.set("你选取颜色RGB是" + color)
            yourColorCanvas["bg"] = color
            
            closeRGB.set("最相近的颜色是" + closest.chn)
            closeColorCanvas["bg"] = closest.rgb
            
            counterRGB.set("  补色搭配：" + colorObj.counter())
            l1["fg"] = colorObj.counter()
            c1, c2, c3 = colorObj.compare()
            compareRGB1.set("对比色搭配1：" + c1)
            compareRGB2.set("对比色搭配2：" + c2)
            compareRGB3.set("对比色搭配3：" + c3)
            l2["fg"] = c1
            l3["fg"] = c2
            l4["fg"] = c3
            
            self.top.destroy()
            
        self.canvas.bind('<ButtonRelease-1>', onLeftButtonUp)
        self.canvas.pack(fill=BOTH, expand=YES)


root = Tk()
root.title("颜色鉴定工具")
root.geometry(geo_str)



''

In [4]:
mainFrame = Frame(root)
mainFrame.pack()
leftFrame = Frame(mainFrame,height = root_height, width = root_width // 2 - 1)
rightFrame = Frame(mainFrame, height = root_height , width = root_width // 2 - 1)
leftFrame.pack(side = "left", padx = 10)
rightFrame.pack(side = "right",padx = 10)


In [5]:
#左边部分

def getScreenColorHandler():
     # 最小化主窗口
    root.state('icon')
    sleep(0.2)
    filename = 'temp.png'
    im = ImageGrab.grab()
    im.save(filename)
    im.close()
    # 显示全屏幕截图
    w = CaptureWindow(filename)
    bt2.wait_window(w.top)
    # recover the main stage
    root.state('normal')
    os.remove(filename)

bt2 = Button(leftFrame, text = "屏幕取色", command = getScreenColorHandler)

bt2.pack()

In [6]:
#右边部分
yourRGB = StringVar()
closeRGB = StringVar()
counterRGB = StringVar()
compareRGB1 = StringVar()
compareRGB2 = StringVar()
compareRGB3 = StringVar()
yourRGB.set("你选取颜色RGB是")
closeRGB.set("最相近的颜色是")


yourColorCanvas = Canvas(rightFrame, bg= "white", height = root_height* 0.3, width= root_height*0.3)
closeColorCanvas = Canvas(rightFrame, bg='white', height = root_height* 0.3, width= root_height*0.3)

Label(rightFrame, textvariable = yourRGB,fg = "black").pack(anchor = 'w')
yourColorCanvas.pack()
Label(rightFrame, textvariable = closeRGB,fg = "black").pack(anchor = 'w')
closeColorCanvas.pack()


l1 = Label(rightFrame, textvariable = counterRGB,fg = "black")
l2 = Label(rightFrame, textvariable = compareRGB1,fg = "black")
l3 = Label(rightFrame, textvariable = compareRGB2,fg = "black")
l4 = Label(rightFrame, textvariable = compareRGB2,fg = "black")
l1.pack(),l2.pack(),l3.pack(),l4.pack()

(None, None, None, None)

In [7]:
root.mainloop()